In [4]:
import os

In [5]:
%pwd

'/home/kulnipun/end_2_end_proj/kidney_disease_classification/research'

In [6]:
os.chdir('../')

In [7]:
%pwd

'/home/kulnipun/end_2_end_proj/kidney_disease_classification'

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_url : str
    local_data_file:Path
    unzip_dir : Path


In [9]:
from src.cnnClassifier.utils.common import read_yaml,create_directories
from src.cnnClassifier.constants import *

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [11]:
import os
import zipfile
import gdown
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size
from src.cnnClassifier.entity.config_entity import DataIngestionConfig

In [12]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [13]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:

    raise e    

[2025-01-29 18:16:02,942: INFO: common: yaml file : config/config.yaml loaded successfully]
[2025-01-29 18:16:02,949: INFO: common: yaml file : params.yaml loaded successfully]
[2025-01-29 18:16:02,952: INFO: common: created directory at : artifacts ]
[2025-01-29 18:16:02,956: INFO: common: created directory at : artifacts/data_ingestion ]
[2025-01-29 18:16:02,958: INFO: 986331348: Downloading data from https://drive.google.com/file/d/1dWCgBxAARec_MF61SrFlIa2BjwENO3gN/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1dWCgBxAARec_MF61SrFlIa2BjwENO3gN
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1dWCgBxAARec_MF61SrFlIa2BjwENO3gN&confirm=t&uuid=f76e97e6-0520-4751-a712-2f5318f4cb71
To: /home/kulnipun/end_2_end_proj/kidney_disease_classification/artifacts/data_ingestion/data.zip
100%|██████████| 941M/941M [01:18<00:00, 11.9MB/s] 

[2025-01-29 18:17:27,417: INFO: 986331348: Downloaded data from https://drive.google.com/file/d/1dWCgBxAARec_MF61SrFlIa2BjwENO3gN/view?usp=sharing into file artifacts/data_ingestion/data.zip]
